In [1]:
!pip install pillow
!pip install pytesseract
!pip install opencv-python-headless
!pip install imutils

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 174kB 2.9MB/s ta 0:00:01
  Running setup.py bdist_wheel for pytesseract ... done
  Stored in directory: /root/.cache/pip/wheels/d5/90/56/ab7b652592da86821293f7cadc1c554aa376a0d57ce414d0a0
Successfully built pytesseract
You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 18.8MB 860kB/s eta 0:00:01
You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Running setup.py bdist_wheel for imutils ... done
  Stored in directory: /root/.cache/pip/wheels/b2/40/59/139d450e68847ef2f27d876d527b13389dac23df0f66526b5d
Successfully built imutils
You are using pip version 18.1, however version 

In [2]:
# in a terminal: add-apt-repository ppa:alex-p/tesseract-ocr
!apt install -y tesseract-ocr-grc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fontconfig fontconfig-config fonts-dejavu-core libcairo2 libdatrie1
  libfontconfig1 libgif7 libgraphite2-3 libharfbuzz0b libicu55 libjbig0
  liblept5 libopenjp2-7 libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0
  libpixman-1-0 libtesseract4 libthai-data libthai0 libtiff5 libwebp5 libx11-6
  libx11-data libxau6 libxcb-render0 libxcb-shm0 libxcb1 libxdmcp6 libxext6
  libxrender1 tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd ucf
The following NEW packages will be installed:
  fontconfig fontconfig-config fonts-dejavu-core libcairo2 libdatrie1
  libfontconfig1 libgif7 libgraphite2-3 libharfbuzz0b libicu55 libjbig0
  liblept5 libopenjp2-7 libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0
  libpixman-1-0 libtesseract4 libthai-data libthai0 libtiff5 libwebp5 libx11-6
  libx11-data libxau6 libxcb-render0 libxcb-shm0 libxcb1 libxdmcp

In [23]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import imutils
import os
import shutil
import pytesseract
import re

def ocrRoster(filename):
    im_rgb = cv.imread('matches/roster/' + filename)
    im_gray = cv.cvtColor(im_rgb, cv.COLOR_BGR2GRAY)
    im_thresh = cv.threshold(im_gray,155,255,cv.THRESH_BINARY_INV)[1]

    lower = np.array([0])
    upper = np.array([15])
    shapeMask = cv.inRange(im_thresh, lower, upper)

    cnts = cv.findContours(shapeMask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    for c in cnts:
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.04 * peri, True)

        if len(approx) == 4:
            (x, y, w, h) = cv.boundingRect(approx)
            ar = w / float(h)

            if ar >= .1 and ar <= .3 and h > 25:
                crop = im_thresh[approx[2][0][1] - 40 : approx[2][0][1] , approx[0][0][0] - 415 : approx[0][0][0] - 10]
                try:
                    text = pytesseract.image_to_string(crop)
                    first = text.split('\n')[0]
                    ascii = first.encode('ascii', 'ignore').strip().split(' ')
                    sans_special = ''
                    for string in ascii:
                        if bool(re.match('[a-zA-Z0-9]+$', string)):
                            sans_special += string + ' '
                    stripped = sans_special.strip()
                    q_check = stripped.split(' ')
                    # Sometimes, the audio icon reads as a 'q'
                    if q_check[-1] == 'q':
                        q_check[-1] = ''
                        stripped = ' '.join(q_check).strip()
                    if stripped:
                        print(filename, stripped)
                except:
                    print('issue parsing text')
                break

for filename in os.listdir('matches/roster'):
    if not filename.startswith('.'):
        ocrRoster(filename)

('129749836.jpg', 'Pilotstream')
issue parsing text
('175385913.jpg', 'Imaaages')
('186365298.jpg', 'sparks2332')
('174690125.jpg', 'Joop')
('147090134.jpg', 'REDQUS3N')
('68132152.jpg', 'Borchardt')
